In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wiki-cc-openweb/wiki_small_50k/wiki_small_50k.csv
/kaggle/input/wiki-cc-openweb/openweb_50k (2)/openweb_50k.csv
/kaggle/input/wiki-cc-openweb/cc_news/cc_news.csv


In [1]:
from datasets import load_dataset, concatenate_datasets

In [2]:
wiki_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/wiki_small_50k/wiki_small_50k.csv")
owt_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/openweb_50k (2)/openweb_50k.csv")
cc_news_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/cc_news/cc_news.csv")
train_ds = concatenate_datasets([wiki_ds["train"],owt_ds["train"],cc_news_ds["train"]])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from tokenizers import ByteLevelBPETokenizer
special_tokens = ['<s>','<pad>','</s>','<unk>','<mask>']
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([x['text'] for x in train_ds], vocab_size = 10_000,min_frequency =2, special_tokens=special_tokens)


In [6]:
tokenizer.save_model("/kaggle/working/")

['/kaggle/working/vocab.json', '/kaggle/working/merges.txt']

In [7]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/")

In [22]:
indices = [0, 2]  # Example indices
subset = train_ds.select(indices)

In [23]:
type(subset)

datasets.arrow_dataset.Dataset

In [31]:
def tokenize(batch):
    result = tokenizer(batch["text"], truncation=True, max_length=254,return_tensors="pt" )
    print(result)
    result = {"input_ids": result["input_ids"].squeeze(0),
    "attention_mask":result["attention_mask"].squeeze(0)}
    print(result)
    return result

train_dataset = subset.map(tokenize, remove_columns=["text"])


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

{'input_ids': tensor([[   0, 9634, 2278, 4204, 2374, 7303, 2278, 4204,  365,  809, 1270,  836,
         5215,  688, 1275, 1939, 2763,   25,   13,  352,  368, 2743, 2875,  400,
           18,  203,  203, 7994,  203, 1238, 7102,  333,  567,  559,  353, 9225,
          784,   83,  348,   77, 1486,   77,   16,  333, 4428,  347, 1171,  325,
         1325,  365,   88,  875,  444, 1274,  288, 1350, 7470,  660, 1309, 3409,
          729,  295,  352,  444, 6709,  601,  286,  457,  303,   18,  631, 1743,
          327,  291, 4820,  376,  267, 4284, 4826,  290, 4193,   16, 4284, 2721,
          290, 4193,  295, 2271,  409, 2721,   16, 7418,   31,  444, 2131, 2533,
          373,  309,  828, 3553, 4864,  295, 1405,  271,   86,  499, 1754, 6139,
           16,  515, 1141, 1720, 7248, 4725,  353, 7303, 2102, 4849,   18,  631,
         1791,  353, 1904,  264,  400, 5550, 1057,  603,  335,   16,  295, 4802,
          384, 2435, 5256,   83,  353, 1904,  264,  400,  315,  363, 1777, 8506,
         2655,

In [20]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 5
})

In [8]:
# Real text


sample_dataset = []
for text in train_ds['text']:
    enc = tokenizer(text, truncation=True, max_length=254, return_tensors="pt")
    sample_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })


In [9]:
from transformers import (
    RobertaConfig,
    RobertaForMaskedLM,

    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

2025-08-23 18:49:34.881904: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755974975.254754      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755974975.353849      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
config = RobertaConfig(
vocab_size = tokenizer.vocab_size,
hidden_size = 256, #Embedding size
num_hidden_layers = 6,#Number of encoder layers
num_attention_heads=4, # Attentions per encoder layer, divisible by 256
intermediate_size = 1024, #FFN layer size
max_position_embeddings=256, #Position Embedding max size = embedding size
type_vocab_size =1,
layer_norm_eps = 1e-5,
initializer_range = 0.02,
pad_token_id = 1,
bos_token_id = 0,
eos_token_id = 2
)
model = RobertaForMaskedLM(config)


In [11]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [ ]:
# 4️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    overwrite_output_dir = False,
    do_train=True,
    save_steps=100,
    logging_steps=100,
    num_train_epochs=15,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_steps = 100,
    logging_dir = "./logs",
    fp16=True,
    save_total_limit=3,
    save_strategy = "steps",
    report_to=["tensorboard"]
)

# 5️⃣ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sample_dataset,
    data_collator=data_collator
)
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,8.935700
200,8.087100
300,7.571400
400,7.391100
500,7.339800
600,7.298800
700,7.265300
800,7.228900
900,7.193200
1000,7.168100


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

In [37]:
import os
os.listdir("/kaggle/working/")

['.virtual_documents', 'checkpoints', 'merges.txt', 'vocab.json', 'logs']

In [39]:
model.save_pretrained("./tiny-roberta")

In [40]:
tokenizer.save_pretrained("./tiny-roberta")

('./tiny-roberta/tokenizer_config.json',
 './tiny-roberta/special_tokens_map.json',
 './tiny-roberta/vocab.json',
 './tiny-roberta/merges.txt',
 './tiny-roberta/added_tokens.json')

In [44]:
from transformers import RobertaTokenizer, RobertaModel

In [46]:
test_model =  RobertaModel.from_pretrained("./tiny-roberta")
test_tokenizer = RobertaTokenizer.from_pretrained("./tiny-roberta")

Some weights of RobertaModel were not initialized from the model checkpoint at ./tiny-roberta and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
test_text = "what are tariffs"

inputs = test_tokenizer(test_text, return_tensors="pt")

In [54]:
import torch
with torch.no_grad():
    outputs = test_model(**inputs)

In [55]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.8621,  0.2247,  2.4125,  ...,  0.4460, -0.4612,  0.4734],
         [-1.1229, -0.6479,  0.5011,  ...,  0.3974,  1.0822,  0.7830],
         [-1.2034,  0.0680,  1.7291,  ...,  1.4815, -0.0974, -0.8972],
         ...,
         [-1.0029,  0.7104,  0.3114,  ..., -0.7080, -0.1807, -0.3309],
         [-0.3993,  0.8103, -0.4200,  ...,  1.1263,  1.6450, -1.1837],
         [-1.0440,  1.0778,  1.1312,  ...,  1.5617, -0.4226,  0.1759]]]), pooler_output=tensor([[-3.3787e-01, -6.4649e-01,  3.5569e-01,  3.2207e-01,  1.5847e-01,
          2.6981e-01,  3.0054e-01,  1.6362e-02,  3.3713e-01,  1.3520e-01,
          3.5580e-01, -1.2581e-01,  2.2033e-02,  3.9121e-01,  1.1300e-01,
         -3.8726e-01, -1.8054e-01,  5.3663e-01, -4.2245e-01,  3.3471e-01,
         -3.4189e-01,  3.9718e-02,  1.4305e-01,  2.5353e-01,  5.8105e-03,
          5.4468e-01,  2.4620e-01,  3.5678e-01, -1.6053e-01,  1.8096e-01,
          2.1180e-01, -7.4367e-02,  

In [70]:
import torch
def get_mean_embed(tokenized_txt):
    with torch.no_grad():
        outputs_exp = test_model(**tokenized_txt)
    last_hidden_state_exp = outputs_exp.last_hidden_state 
    cls_embedding_exp = last_hidden_state_exp[:, 0, :]     # [CLS] token representation
    mean_embedding_exp = last_hidden_state_exp.mean(dim=1)
    return mean_embedding_exp

In [75]:
def findsimilaritybetweendocs(doc1, doc2):
    #Tokenize the docs
    doc1_tk = test_tokenizer(doc1, return_tensors="pt")
    doc2_tk = test_tokenizer(doc2, return_tensors="pt")
    #Find the embeddings for the doc
    doc1_emb = get_mean_embed(doc1_tk)
    doc2_emb = get_mean_embed(doc2_tk)
    cos_sim = F.cosine_similarity(doc1_emb, doc2_emb)
    return cos_sim.item()
    

In [76]:
findsimilaritybetweendocs("What is Tarif","Tariffs are taxes imposed by governments on imported goods and services, used to influence trade, protect domestic industries, and generate revenue.")

0.8751513957977295

In [80]:
findsimilaritybetweendocs("What is the capital of France?","The capital of France is Paris.")

0.9255729913711548

In [86]:
findsimilaritybetweendocs("What is the capital of France?","I gIood I")

0.8300344347953796

In [ ]:
import json
import matplotlib.pyplot as plt

# Path to your trainer_state.json (inside output_dir)
state_file = "./checkpoints/trainer_state.json"

with open(state_file, "r") as f:
    trainer_state = json.load(f)

# Extract loss and step history
steps = []
losses = []

for log in trainer_state.get("log_history", []):
    if "loss" in log:
        steps.append(log["step"])
        losses.append(log["loss"])

# Plot
plt.figure(figsize=(8, 5))
plt.plot(steps, losses, marker="o")
plt.xlabel("Step")
plt.ylabel("Training Loss")
plt.title("Training Loss over Steps")
plt.grid(True)
plt.show()


In [ ]:
import json
import matplotlib.pyplot as plt

# Path to your trainer_state.json
state_file = "./checkpoints/trainer_state.json"

with open(state_file, "r") as f:
    trainer_state = json.load(f)

# Extract steps, loss, and learning rate
steps = []
losses = []
lrs = []

for log in trainer_state.get("log_history", []):
    if "loss" in log and "learning_rate" in log:
        steps.append(log["step"])
        losses.append(log["loss"])
        lrs.append(log["learning_rate"])

# Plot with dual axes
fig, ax1 = plt.subplots(figsize=(9, 5))

color = "tab:blue"
ax1.set_xlabel("Step")
ax1.set_ylabel("Training Loss", color=color)
ax1.plot(steps, losses, color=color, marker="o", label="Loss")
ax1.tick_params(axis="y", labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()  # second y-axis
color = "tab:red"
ax2.set_ylabel("Learning Rate", color=color)
ax2.plot(steps, lrs, color=color, linestyle="--", label="Learning Rate")
ax2.tick_params(axis="y", labelcolor=color)

fig.suptitle("Training Loss and Learning Rate over Steps")
plt.show()
